In [1]:
import numpy as np
import glob, sys, time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import EnglishStemmer

In [2]:
FOLDER = '../enron1/'
HAM_FOLDER = 'ham/'
SPAM_FOLDER = 'spam/'

HAM = 0
SPAM = 1

HAM_LIST = glob.glob(FOLDER + HAM_FOLDER + '*.txt')
SPAM_LIST = glob.glob(FOLDER + SPAM_FOLDER + '*.txt')

In [19]:
class Classifier:

    def __init__(self, ham_list, spam_list):
        self.ham_list = ham_list
        self.spam_list = spam_list
        self.email_list = ham_list + spam_list
        self.N_HAM = np.size(ham_list)
        self.N_SPAM = np.size(spam_list)
        self.N = np.asarray([self.N_HAM, self.N_SPAM])
        self.label = np.asarray([HAM]* self.N_HAM + [SPAM]* self.N_SPAM)
        
        # number of training docs in ham and spam folder
        # default to be 80% of total
        self.p_training = 0.8
        self.N_TRAINING = np.asarray([int(np.floor(self.N_HAM * 0.8)), int(np.floor(self.N_SPAM * 0.8))])
        self.N_TESTING = self.N - self.N_TRAINING
        
        self.training_X = None
        self.training_label = None
        self.testing_X = None
        self.testing_label = None
        self.result = None
        
        # container for vocabulary list
        self.vocab = []
        self.nvocab = 0
        
    def nwords(self, X):
        '''return the number of distinct words in input matrix X by counting the non-empty columns in X
            parameters:
                X: 2d numpy array'''
        return np.count_nonzero(X.sum(axis = 0))
    
    def vectorize(self, p_training = None):
        print('vectorizing the emails...')
        start_time = time.time()
        
        if p_training != None:
            self.p_training = p_training
        # else p_training = 0.8 by default
        
        # [number of ham in training, number of spam in training]
        self.N_TRAINING = np.asarray([int(np.floor(self.N_HAM * self.p_training)),
                                      int(np.floor(self.N_SPAM * self.p_training))])
        # [number of ham in testing, number of spam in testing]
        self.N_TESTING = self.N - self.N_TRAINING
        
        # word stemming
        stemmer = EnglishStemmer()
        analyzer = CountVectorizer(input = 'filename', decode_error = 'ignore').build_analyzer()
        def stemmed_words(doc):
            return (stemmer.stem(w) for w in analyzer(doc))
        
        training = CountVectorizer(input = 'filename', decode_error = 'ignore', analyzer = stemmed_words, 
                                   max_df = 0.95, min_df = 5)
        self.training_X = training.fit_transform(self.ham_list[:self.N_TRAINING[HAM]] + self.spam_list[:self.N_TRAINING[SPAM]]).toarray()
        
        # get the vocabulary list from training data
        self.vocab = training.get_feature_names()
        self.nvocab = np.size(self.vocab)

        testing = CountVectorizer(input = 'filename', vocabulary = self.vocab, decode_error = 'ignore')
        self.testing_X = testing.fit_transform(self.ham_list[-self.N_TESTING[HAM]:] + self.spam_list[-self.N_TESTING[SPAM]:]).toarray()
        
        # create the label arrays
        self.training_label = np.asarray([HAM] * self.N_TRAINING[HAM] + [SPAM] * self.N_TRAINING[SPAM])
        self.testing_label = np.asarray([HAM] * self.N_TESTING[HAM] + [SPAM] * self.N_TESTING[SPAM])
        
        print('vectorizing took %.2f s' % (time.time() - start_time))
        return self.training_X, self.training_label, self.testing_X, self.testing_label
    
    def get_ham(self, X, label):
        return X[np.where(label == HAM)]
    
    def get_spam(self, X, label):
        return X[np.where(label == SPAM)]
                
    def accuracy(self, result = None):
        if result != None:
            self.result = result
        return np.mean(self.result == self.testing_label) # number of correct predictions / total testing cases

    def naive_bayes(self, p_training = None):
        if p_training != None:
            # re-vectorize the data
            self.vectorize(p_training)
        # we use the multinomial naive bayes model from 
        # https://web.stanford.edu/class/cs124/lec/naivebayes.pdf
        def get_prior():
            '''get the prior of for the Naive Bayes method which will be
            [fraction of ham emails, fraction of spam emails]'''
            prior = self.N_TRAINING / self.N_TRAINING.sum()
            return prior

        def get_conditionals():
            # split the traning data by label
            training_ham = self.training_X[:self.N_TRAINING[HAM]]
            training_spam = self.training_X[-self.N_TRAINING[SPAM]:]

            # conditionals with Laplace smoothing
            con_ham = (training_ham.sum(axis = 0) + 1) / (self.nwords(training_ham) + self.nvocab)
            con_spam = (training_spam.sum(axis = 0) + 1) / (self.nwords(training_spam) + self.nvocab)
            conditionals = np.asarray([con_ham, con_spam])
            return conditionals

        print('cross validating...')
        start_time = time.time()

        prior = get_prior()
        conditionals = get_conditionals()
        # start applying labels to our testing data!
        self.result = np.empty(self.N_TESTING.sum()) # the results of our classifier
        for i in np.arange(self.N_TESTING.sum()):
            # use log likelihood for easier calculation
            loglike_ham = np.dot(np.log(conditionals[HAM]), self.testing_X[i]) + np.log(prior[HAM])
            loglike_spam = np.dot(np.log(conditionals[SPAM]), self.testing_X[i]) + np.log(prior[SPAM])
            self.result[i] = HAM if loglike_ham > loglike_spam else SPAM
        print('testing took %.2f s' % (time.time() - start_time))
        return self.result

In [24]:
test = Classifier(HAM_LIST, SPAM_LIST)

In [21]:
# train_arr, train_arr_label, test_arr, test_arr_label = test.vectorize(0.8)

vectorizing the emails...
vectorizing took 4.02 s


In [25]:
result = test.naive_bayes(0.9)

vectorizing the emails...
vectorizing took 4.49 s
cross validating...
testing took 0.14 s


In [26]:
test.accuracy()

0.8166023166023166